In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
df = pd.read_hdf('../metadata.h5')
df.head()

,date_added,date_modified,guid,id,meta_info,name,parents,type,url,keyword,img_url,children_id
0,13202479594197706,13242831177794171,00000000-0000-4000-a000-000000000002,1,,書籤列,[1],folder,,[],,"[1884, 1891, 1897, 1903, 1906, 1907, 1908, 190..."
1,13233003241034750,13233003241037911,f0b52077-6793-498a-b8eb-462dfc400922,1884,,2019-nCoV,"[1, 1884]",folder,,"[2019, nCoV]",,"[1885, 1886, 1888, 1890]"
2,13226238181043741,,7f8a21b9-b559-44ab-b711-c41800d188d1,1885,,Coronavirus_Taiwan,"[1, 1884, 1885]",url,https://viator.maps.arcgis.com/apps/opsdashboa...,"[Coronavirus, Taiwan]",,[]
3,13226407607803828,,24a7d7be-2a87-4481-b157-3773db2f4eab,1886,,2019-nCoV | 武漢肺炎地圖,"[1, 1884, 1886]",url,https://kiang.github.io/2019-nCoV/,"[2019, nCoV, 武漢, 地圖, 肺炎]",http://kiang.github.io/2019-nCoV/og_image.png,[]
4,13227360115873492,,a09ae3f0-3a5f-417c-ad10-423d35e155f3,1888,,程序員硬核勸告：現在還不是出門的時候 - 幫趣,"[1, 1884, 1888]",url,https://bangqu.com/a7wXH5.html,"[硬核, 勸告, 現在還, 出門, 時候, 幫趣, 程序]",https://d2ndd3gtcc6iwc.cloudfront.net/liang/20...,[]


In [3]:
keywords = np.concatenate(df['keyword'])
keywords_unique = np.unique(keywords)
print(keywords_unique.shape)
keywords_unique

(3572,)


array(['....', '0.14', '0.22', ..., '黨太嚇', '鼓勵', '齊震宇'], dtype='<U32')

In [4]:
tags = pd.Series(keywords_unique)
tags.head()

0    ....
1    0.14
2    0.22
3      00
4     001
dtype: object

In [5]:
def ids_which_target_is_in_keyword(target):
    has_target_indicator = df['keyword'].apply(lambda list_: target in list_)
    ids = df.loc[has_target_indicator, 'id'].values
    return ids

bookmark_ids = tags.progress_apply(ids_which_target_is_in_keyword)

100%|█████████████████████████████████████████████████████████████████████████████| 3572/3572 [00:06<00:00, 513.96it/s]


In [6]:
tag2ids = pd.Series(bookmark_ids.values, index=tags)
tag2ids

....                                          [3000, 3180]
0.14                                                [2545]
0.22                                                [3185]
00                                                  [2153]
001                                           [1962, 3112]
01                                            [3156, 3235]
010                                           [3166, 3137]
010617                                              [3127]
013                                                 [3166]
017                                                 [3045]
019                                                 [3137]
02                                [2089, 2140, 3261, 3262]
0215                                                [3048]
022                                                 [3038]
03                                            [2997, 2929]
039                                                 [3108]
041                                                 [303

In [10]:
arr = np.zeros(df.shape)
arr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
for i, li in enumerate(tag2ids[:10]):
    for j in li:
        print(type(i), type(j))
        arr[i, j] = 1

<class 'int'> <class 'str'>


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [39]:
#tag2ids.to_hdf('../static/tag_series/word_tokenized.h5', key='data')

C:\Users\adam8\Anaconda3\lib\site-packages\pandas\core\generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->values] [items->None]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


# create matrix

In [14]:
def ids_which_target_is_in_keyword(target):
    has_target_indicator = df['keyword'].apply(lambda list_: target in list_)
    return np.where(has_target_indicator)[0]

bookmark_iloc = tags.progress_apply(ids_which_target_is_in_keyword)

100%|█████████████████████████████████████████████████████████████████████████████| 3572/3572 [00:05<00:00, 667.13it/s]


In [15]:
tag2ids = pd.Series(bookmark_iloc.values, index=tags)
tag2ids

....                                    [544, 764]
0.14                                          [23]
0.22                                         [770]
00                                           [478]
001                                     [132, 526]
01                                      [641, 805]
010                                     [624, 653]
010617                                       [620]
013                                          [624]
017                                          [594]
019                                          [653]
02                            [284, 336, 484, 486]
0215                                         [538]
022                                          [529]
03                                      [567, 782]
039                                          [560]
041                                          [529]
042614                                       [539]
043                                     [577, 591]
049                            

In [20]:
arr = np.zeros([df.shape[0], tags.shape[0]])
arr.shape

(919, 3572)

In [23]:
for i, li in tqdm(enumerate(tag2ids)):
    for j in li:
        arr[j, i] = 1

3572it [00:00, 146973.66it/s]


In [29]:
mat = pd.DataFrame(arr)
mat.corr()

,0,1,2,3,4,5,6,7,8,9,...,3562,3563,3564,3565,3566,3567,3568,3569,3570,3571
0,1.000000,-0.002181,-0.002181,-0.002181,-0.002181,-0.003454,-0.002181,-0.003088,-0.003454,-0.005594,...,-0.001541,-0.004376,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541
1,-0.002181,1.000000,-0.002181,0.498909,-0.002181,-0.003454,-0.002181,-0.003088,-0.003454,-0.005594,...,-0.001541,-0.004376,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541
2,-0.002181,-0.002181,1.000000,-0.002181,-0.002181,-0.003454,-0.002181,-0.003088,-0.003454,-0.005594,...,-0.001541,-0.004376,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541
3,-0.002181,0.498909,-0.002181,1.000000,-0.002181,-0.003454,-0.002181,-0.003088,-0.003454,-0.005594,...,-0.001541,-0.004376,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541
4,-0.002181,-0.002181,-0.002181,-0.002181,1.000000,-0.003454,-0.002181,-0.003088,-0.003454,-0.005594,...,-0.001541,-0.004376,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541
5,-0.003454,-0.003454,-0.003454,-0.003454,-0.003454,1.000000,-0.003454,-0.004890,-0.005470,-0.008860,...,-0.002441,-0.006931,-0.002441,-0.002441,-0.002441,-0.002441,-0.002441,-0.002441,-0.002441,-0.002441
6,-0.002181,-0.002181,-0.002181,-0.002181,-0.002181,-0.003454,1.000000,-0.003088,0.313983,-0.005594,...,-0.001541,-0.004376,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541,-0.001541
7,-0.003088,-0.003088,-0.003088,-0.003088,-0.003088,-0.004890,-0.003088,1.000000,-0.004890,-0.007920,...,-0.002182,-0.006196,-0.002182,-0.002182,-0.002182,-0.002182,-0.002182,-0.002182,-0.002182,-0.002182
8,-0.003454,-0.003454,-0.003454,-0.003454,-0.003454,-0.005470,0.313983,-0.004890,1.000000,0.116403,...,-0.002441,-0.006931,-0.002441,-0.002441,-0.002441,-0.002441,-0.002441,-0.002441,-0.002441,-0.002441
9,-0.005594,-0.005594,-0.005594,-0.005594,-0.005594,-0.008860,-0.005594,-0.007920,0.116403,1.000000,...,-0.003954,-0.011225,-0.003954,-0.003954,-0.003954,-0.003954,-0.003954,-0.003954,-0.003954,-0.003954
